In [20]:
import pandas as pd

In [21]:
import time

In [22]:
import datetime as dt

In [23]:
import pytz
eastern = pytz.timezone('US/Eastern')

In [24]:
pd.set_option('display.max_columns', None)

In [25]:
def clean_file(df):
    df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
    for i in range(len(df)):
        start = df.at[i, 'Published Start']
        if start[-1] == 'a' or start[0:2] == '12':
            if len(start.split(':')[0]) == 1:
                df.at[i, 'Published Start'] = '0%s' %(start[:-1])
            else:
                df.at[i, 'Published Start'] = start[:-1]
        elif start == 'noon':
            df.at[i, 'Published Start'] = '12:00'
        else:
            hour = int(start.split(':')[0])
            hour += 12
            rest = start.split(':')[1]
            df.at[i, 'Published Start'] = '%d:%s' %(hour,rest[0:2])


        end = df.at[i, 'Published End']
        if end[-1] == 'a' or end[0:2] == '12':
            if len(end.split(':')[0]) == 1:
                df.at[i, 'Published End'] = '0%s' %(end[:-1])
            else:
                df.at[i, 'Published End'] = end[:-1]
        elif end == 'noon':
            df.at[i, 'Published End'] = '12:00'
        else:
            hour = int(end.split(':')[0])
            hour += 12
            rest = end.split(':')[1]
            df.at[i, 'Published End'] = '%d:%s' %(hour,rest[0:2])

        df.at[i, 'Building'] = df.at[i, 'Location'].split()[0]
        df.at[i, 'Room'] = df.at[i, 'Location'].split()[1]

def combine_time(df):
    for i in range(len(df)):
        df.at[i, "Start Time"] = df.at[i, "Date"] + "T" + df.at[i, "Published Start"]
        df.at[i, "End Time"] = df.at[i, "Date"] + "T" + df.at[i, "Published End"]

def convert_iso(df):
    df["Start Time"] = pd.to_datetime(df["Start Time"])
    df["End Time"] = pd.to_datetime(df["End Time"])

def to_epoch(df):
    for i in range(len(df)):
        start = eastern.localize(df.at[i, "Start Time"])
        end = eastern.localize(df.at[i, "End Time"])
        df.at[i, "Start Time"] = start.timestamp()
        df.at[i, "End Time"] = end.timestamp()

def drop_useless_cols(df):
    df = df[['Name', 'Section', 'Start Time', 'End Time', 'Building', 'Room', 'Capacity', 'Address', 'Lat', 'Long']]

def insert_address(df, address):
    for i in range(len(address)):
        arr = address.at[i, "Combined"].split(', ')
        address.at[i, "Lat"] = arr[0]
        address.at[i, "Long"] = arr[1]
    address = address.drop(["Combined"], axis=1)
    df = pd.merge(df, address, on="Building")
    return df

def insert_beg_end(df):
    for date in set(df["Date"]):
        for classroom in set(df["Location"]):
            df.loc[len(df)] = [pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, date, '06:00', '6:00', classroom, df[df.Location == classroom].iloc[0]['Capacity'], pd.NA, pd.NA, pd.NA, pd.NA, df[df.Location == classroom].iloc[0]['Building'], df[df.Location == classroom].iloc[0]['Room'], df[df.Location == classroom].iloc[0]['Address'], df[df.Location == classroom].iloc[0]['Lat'], df[df.Location == classroom].iloc[0]['Long']]
            df.loc[len(df)] = [pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, date, '22:00', '22:00', classroom, df[df.Location == classroom].iloc[0]['Capacity'], pd.NA, pd.NA, pd.NA, pd.NA, df[df.Location == classroom].iloc[0]['Building'], df[df.Location == classroom].iloc[0]['Room'], df[df.Location == classroom].iloc[0]['Address'], df[df.Location == classroom].iloc[0]['Lat'], df[df.Location == classroom].iloc[0]['Long']]

In [26]:
df = pd.read_csv("meetings.csv")

In [27]:
address = pd.read_csv("combined_address.csv")

In [28]:
clean_file(df)

In [29]:
df = insert_address(df, address)

In [30]:
insert_beg_end(df)

In [31]:
df_copy = df.copy()

In [32]:
df = df_copy.copy()

In [33]:
combine_time(df)

In [34]:
convert_iso(df)

In [35]:
drop_useless_cols(df)

In [36]:
to_epoch(df)

In [37]:
df.head()

,Name,Section,Type,Title,Note,Date,Published Start,Published End,Location,Capacity,Instructor / Organization,Email,Requested Services,Approved,Building,Room,Address,Lat,Long,Start Time,End Time
0,AAE 20300,10001-001,Lecture,Aeromechanics I,Evening Exams Required.,04/17/2023,10:30,11:20,ME 2061,96,"Hassan, Hashim (Instr)",hassan18@purdue.edu,NaN,09/06/2022,ME,2061,585 Purdue Mall,40.4283678200671,-86.9129040439333,1681741800.0,1681744800.0
1,AAE 20300,10001-001,Lecture,Aeromechanics I,Evening Exams Required.,04/19/2023,10:30,11:20,ME 2061,96,"Hassan, Hashim (Instr)",hassan18@purdue.edu,NaN,09/06/2022,ME,2061,585 Purdue Mall,40.4283678200671,-86.9129040439333,1681914600.0,1681917600.0
2,AAE 20300,10001-001,Lecture,Aeromechanics I,Evening Exams Required.,04/21/2023,10:30,11:20,ME 2061,96,"Hassan, Hashim (Instr)",hassan18@purdue.edu,NaN,09/06/2022,ME,2061,585 Purdue Mall,40.4283678200671,-86.9129040439333,1682087400.0,1682090400.0
3,AAE 20300,10001-001,Lecture,Aeromechanics I,Evening Exams Required.,04/24/2023,10:30,11:20,ME 2061,96,"Hassan, Hashim (Instr)",hassan18@purdue.edu,NaN,09/06/2022,ME,2061,585 Purdue Mall,40.4283678200671,-86.9129040439333,1682346600.0,1682349600.0
4,AAE 20300,10001-001,Lecture,Aeromechanics I,Evening Exams Required.,04/26/2023,10:30,11:20,ME 2061,96,"Hassan, Hashim (Instr)",hassan18@purdue.edu,NaN,09/06/2022,ME,2061,585 Purdue Mall,40.4283678200671,-86.9129040439333,1682519400.0,1682522400.0


In [38]:
df = df.drop(['Name', 'Section', 'Type', 'Title', 'Note', 'Approved', 'Instructor / Organization',
       'Email', 'Requested Services'], axis=1)

In [39]:
sorted_df = df.sort_values("Published Start")

In [40]:
date_list = []
for d_index, date in enumerate(sorted(list(set(df["Date"])))):
    class_list = list()
    for c_index, classroom in enumerate(sorted(list(set(sorted_df["Location"])))):
        temp_df = sorted_df[(sorted_df["Location"] == classroom) & (sorted_df["Date"] == date)].reset_index(drop=True)
        if len(temp_df) == 0:
            continue
        class_list.append(temp_df)
    date_list.append(class_list)

In [41]:
# things = list()

In [42]:
# thing = pd.DataFrame(things, columns=['Date', 'Published Start', 'Published End', 'Location', 'Capacity',
#        'Building', 'Room', 'Address', 'Lat', 'Long', 'Start Time',
#        'End Time'])

In [43]:
# day = 0
# room = 0
# temp_things = list()
# temp_df = date_list[0][0].values
# for i in range(1, len(date_list[day][room])):
#     print(i)
#     temp_row = temp_df[0].copy()
#     temp_row[10] = int(temp_df[i - 1][11])
#     temp_row[11] = int(temp_df[i][10])
#     temp_start = time.gmtime(temp_row[10])
#     temp_end = time.gmtime(temp_row[11])
#     temp_row[1] = '{0:02d}:{1:02d}'.format(temp_start[3], temp_start[4])
#     temp_row[2] = '{0:02d}:{1:02d}'.format(temp_end[3], temp_end[4])
#     temp_things.append(temp_row)

# temp_things = pd.DataFrame(temp_things, columns=['Date', 'Published Start', 'Published End', 'Location', 'Capacity',
#        'Building', 'Room', 'Address', 'Lat', 'Long', 'Start Time',
#        'End Time'])

In [44]:
final = list()
for day in range(len(date_list)):
    day_list = list()
    for room in range(len(date_list[day])):
        room_list = list()
        temp_df = date_list[day][room].values
        for i in range(1, len(date_list[day][room])):
            temp_row = temp_df[0].copy()
            temp_row[10] = int(temp_df[i - 1][11])
            temp_row[11] = int(temp_df[i][10])
            temp_start = time.localtime(temp_row[10])
            temp_end = time.localtime(temp_row[11])
            temp_row[1] = '{0:02d}:{1:02d}'.format(temp_start[3], temp_start[4])
            temp_row[2] = '{0:02d}:{1:02d}'.format(temp_end[3], temp_end[4])
            room_list.append(temp_row)
        thing = pd.DataFrame(room_list, columns=['Date', 'Published Start', 'Published End', 'Location', 'Capacity', 'Building', 'Room', 'Address', 'Lat', 'Long', 'Start Time','End Time'])
        day_list.append(thing)
    final.append(day_list)      

In [45]:
final_df = pd.DataFrame()
for i in range(len(final)):
    temp_concat = pd.concat(final[i])
    final_df = pd.concat([final_df, temp_concat])

In [48]:
final_df.head()

,Date,Published Start,Published End,Location,Capacity,Building,Room,Address,Lat,Long,Start Time,End Time
0,04/17/2023,06:00,10:30,ABE B053,32,ABE,B053,225 S University Street,40.42184583828238,-86.91660228811303,1681725600,1681741800
1,04/17/2023,11:20,11:30,ABE B053,32,ABE,B053,225 S University Street,40.42184583828238,-86.91660228811303,1681744800,1681745400
2,04/17/2023,12:20,12:30,ABE B053,32,ABE,B053,225 S University Street,40.42184583828238,-86.91660228811303,1681748400,1681749000
3,04/17/2023,13:20,13:30,ABE B053,32,ABE,B053,225 S University Street,40.42184583828238,-86.91660228811303,1681752000,1681752600
4,04/17/2023,14:20,14:30,ABE B053,32,ABE,B053,225 S University Street,40.42184583828238,-86.91660228811303,1681755600,1681756200


In [47]:
final_df.to_csv('sprint3.csv', index=False, header=False)